# Characterizing Lidar Point-Cloud Adversities Using a Vector Field Visualization

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# import scipy.io as sio
# import datetime
# import trimesh 
# import time
import math
from vedo import *
# from trimesh import creation, transformations
from ipyvtklink.viewer import ViewInteractiveWidget
# import open3d as o3d
# from pyndt import NDT

# # Limit GPU memory, if running notebook on GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# if gpus:
#     try:
#         memlim = 2*1024
#         tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
#     except RuntimeError as e:
#         print(e)

2024-07-30 12:51:25.464136: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 12:51:25.488974: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 12:51:25.488990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 12:51:25.490034: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 12:51:25.494041: I tensorflow/core/platform/cpu_feature_gua

In [2]:
# Import experimental data
ptCloud1_fn = "/home/daniel-choate/ASAR/s1/ErrorVectorFields/Cloud1_ExpD.txt"
ptCloud1 = np.loadtxt(ptCloud1_fn)
print(ptCloud1)
ptCloud10_fn = "/home/daniel-choate/ASAR/s1/ErrorVectorFields/Cloud10_ExpD.txt"
ptCloud10 = np.loadtxt(ptCloud10_fn)
print(ptCloud10)

# Plot scans using vedo 
plt1 = Plotter(N = 1, axes = 4, bg = (1,1,1), interactive = True)
disp = []
disp.append(Points(ptCloud1, c = '#0072BD', r = 2, alpha = 1)) #+ np.array([-13,-13,3]) #0343DF
disp.append(Points(ptCloud10, c = 'red', r = 2, alpha = 1)) #+ np.array([-13,-13,3])

print(disp)

plt1.show(disp, "Fig 1.1")
ViewInteractiveWidget(plt1.window)

[[        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 ...
 [-0.1363716  15.946853   -4.2731023 ]
 [        nan         nan         nan]
 [-0.02500946 15.924233   -4.266891  ]]
[[            nan             nan             nan]
 [            nan             nan             nan]
 [            nan             nan             nan]
 ...
 [            nan             nan             nan]
 [-7.01405481e-02  1.60747852e+01 -4.30726671e+00]
 [-1.11888228e-02  1.60208416e+01 -4.29277277e+00]]
[<vedo.pointcloud.Points object at 0x7289e5b1f910>, <vedo.pointcloud.Points object at 0x7289e6eb0400>]


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 60
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 60
pci id for fd 60: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 

ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=800)

In [3]:
# Convert .txt files to .xyz files to be match open3d format

# Replace with your input and output file paths
input_file = ptCloud1_fn
output_file1 = "/home/daniel-choate/ASAR/s1/ErrorVectorFields/Cloud1_ExpD.xyz"

# Read data from .txt file
with open(input_file, 'r') as f:
    lines = f.readlines()

# Extract XYZ coordinates and write to .xyz file
with open(output_file1, 'w') as f:
    for line in lines:
        if line.strip():  # Check if line is not empty
            # Assuming each line contains XYZ coordinates separated by spaces or commas
            parts = line.strip().split()
            if len(parts) >= 3:
                x, y, z = parts[:3]  # Extract XYZ coordinates
                f.write(f"{x} {y} {z}\n")  # Write XYZ coordinates in .xyz format

# Replace with your input and output file paths
input_file = ptCloud10_fn
output_file10 = "/home/daniel-choate/ASAR/s1/ErrorVectorFields/Cloud10_ExpD.xyz"

# Read data from .txt file
with open(input_file, 'r') as f:
    lines = f.readlines()

# Extract XYZ coordinates and write to .xyz file
with open(output_file10, 'w') as f:
    for line in lines:
        if line.strip():  # Check if line is not empty
            # Assuming each line contains XYZ coordinates separated by spaces or commas
            parts = line.strip().split()
            if len(parts) >= 3:
                x, y, z = parts[:3]  # Extract XYZ coordinates
                f.write(f"{x} {y} {z}\n")  # Write XYZ coordinates in .xyz format


In [4]:
# Align scans using NDT

pcd1 = o3d.io.read_point_cloud(output_file1)
pcd10 = o3d.io.read_point_cloud(output_file10)

# voxel_size = 0.05
# pcd1_down = pcd1.voxel_down_sample(voxel_size=voxel_size)
# pcd2_down = pcd2.voxel_down_sample(voxel_size=voxel_size)

o3d.visualization.draw_geometries([pcd1, pcd10], point_show_normal=False, mesh_show_wireframe=True)
print(pcd1)
print(pcd10)

source = pcd1
target = pcd10

# Perform NDT registration
# result = o3d.pipelines.registration.registration_ndt(pcd1, pcd10)
# result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(source, target, o3d.pipelines.registration.FastGlobalRegistrationOption())

# # Get the transformation matrix
# T = registration.transformation

# Apply transformation to cloud2 to align it with cloud1
# cloud10_aligned = pcd10.transform(T)

NameError: name 'o3d' is not defined

In [ ]:
# Import previously aligned scan 10 using NDT 

ptCloud10_fn = "/home/daniel-choate/ASAR/s1/ErrorVectorFields/Cloud10Aligned_ExpD.txt"
ptCloud10 = np.loadtxt(ptCloud10_fn)
print(ptCloud10)

# Plot newly aligned clouds
# Plot scans using vedo 
plt2 = Plotter(N = 1, axes = 4, bg = (1,1,1), interactive = True)
disp2 = []
disp2.append(Points(ptCloud1, c = '#0072BD', r = 2, alpha = 1)) #+ np.array([-13,-13,3]) #0343DF
disp2.append(Points(ptCloud10, c = 'red', r = 2, alpha = 1)) #+ np.array([-13,-13,3])

print(disp2)

plt2.show(disp2, "Fig 1.2")
ViewInteractiveWidget(plt2.window)
